# Graph Processor with Agent-as-Tool and invocation_state
Agent를 Tool로 사용하면서 invocation_state에 그래프 데이터를 수집

In [3]:
from strands import Agent, tool, ToolContext
from strands.models import BedrockModel

bedrock_model = BedrockModel(
    model_id="apac.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="ap-northeast-2",
    temperature=0.3
)

## 1. 기본 데이터 수집 Tools

In [4]:
@tool(context=True)
def add_node(entity: str, entity_type: str, tool_context: ToolContext) -> str:
    """그래프에 노드 추가"""
    nodes = tool_context.invocation_state.get("nodes", [])
    nodes.append({"entity": entity, "type": entity_type})
    tool_context.invocation_state["nodes"] = nodes
    print(f"✓ Added node: {entity} ({entity_type})")
    return f"Added: {entity}"

@tool(context=True)
def add_edge(source: str, target: str, relation: str, tool_context: ToolContext) -> str:
    """그래프에 엣지 추가"""
    edges = tool_context.invocation_state.get("edges", [])
    edges.append({"source": source, "target": target, "relation": relation})
    tool_context.invocation_state["edges"] = edges
    print(f"✓ Added edge: {source} --[{relation}]--> {target}")
    return f"Added: {source} -> {target}"

## 2. Agent-as-Tool 구조

In [5]:
@tool(context=True)
def entity_extractor(text: str, tool_context: ToolContext) -> str:
    """텍스트에서 엔티티를 추출하는 Agent"""
    agent = Agent(
        name="entity_extractor",
        model=bedrock_model,
        system_prompt="Extract entities from text and add them using add_node tool.",
        tools=[add_node]
    )
    response = agent(text, invocation_state=tool_context.invocation_state)
    return str(response)

@tool(context=True)
def relation_extractor(text: str, tool_context: ToolContext) -> str:
    """텍스트에서 관계를 추출하는 Agent"""
    agent = Agent(
        name="relation_extractor",
        model=bedrock_model,
        system_prompt="Extract relationships from text and add them using add_edge tool.",
        tools=[add_edge]
    )
    response = agent(text, invocation_state=tool_context.invocation_state)
    return str(response)

@tool(context=True)
def graph_builder(text: str, tool_context: ToolContext) -> str:
    """엔티티와 관계를 모두 추출하는 통합 Agent"""
    agent = Agent(
        name="graph_builder",
        model=bedrock_model,
        system_prompt="Build graph by extracting entities first, then relationships.",
        tools=[entity_extractor, relation_extractor]
    )
    response = agent(text, invocation_state=tool_context.invocation_state)
    return str(response)

## 3. 그래프 조회 Tool

In [6]:
@tool(context=True)
def show_graph(tool_context: ToolContext) -> str:
    """수집된 그래프 데이터 출력"""
    nodes = tool_context.invocation_state.get("nodes", [])
    edges = tool_context.invocation_state.get("edges", [])
    
    result = f"\n=== Graph Summary ===\n"
    result += f"Nodes ({len(nodes)}): {[n['entity'] for n in nodes]}\n"
    result += f"Edges ({len(edges)}): {[(e['source'], e['relation'], e['target']) for e in edges]}"
    
    print(result)
    return result

## 4. Orchestrator 실행

In [7]:
orchestrator = Agent(
    name="orchestrator",
    model=bedrock_model,
    system_prompt="Build knowledge graph using graph_builder, then show results with show_graph.",
    tools=[graph_builder, show_graph]
)

result = orchestrator(
    "AWS Lambda integrates with Amazon S3 and DynamoDB. S3 stores data that Lambda processes.",
    invocation_state={"nodes": [], "edges": []}
)

print("\n=== Final Result ===")
print(result)

I'll build a knowledge graph from the information about AWS Lambda and its integrations with Amazon S3 and DynamoDB.
Tool #1: graph_builder
I'll help you build a graph by first extracting entities and then relationships from your text about AWS Lambda integration.
Tool #1: entity_extractor
I'll extract the entities from the text and add them to the graph using the add_node tool.
Tool #1: add_node

Tool #2: add_node

Tool #3: add_node

Tool #4: add_node

Tool #5: add_node

Tool #6: add_node
✓ Added node: AWS Lambda (service)
✓ Added node: Amazon S3 (service)
✓ Added node: DynamoDB (service)
✓ Added node: S3 (service)
✓ Added node: Lambda (service)
✓ Added node: data (concept)
I've successfully extracted and added the following entities from the text:

**Services:**
- AWS Lambda
- Amazon S3
- DynamoDB
- S3 (as a separate reference)
- Lambda (as a separate reference)

**Concepts:**
- data

These entities represent the key components mentioned in your text about AWS Lambda's integration wi

## 구조 설명

```
Orchestrator
  ↓ invocation_state={nodes:[], edges:[]}
graph_builder (Agent-as-Tool)
  ↓ invocation_state 전달
  ├─ entity_extractor (Agent-as-Tool)
  │   ↓ invocation_state 전달
  │   └─ add_node (Tool) → nodes 배열에 추가
  │
  └─ relation_extractor (Agent-as-Tool)
      ↓ invocation_state 전달
      └─ add_edge (Tool) → edges 배열에 추가
  ↓
show_graph (Tool) → invocation_state 조회
```